In [66]:
import xarray as xr
from dask.distributed import Client

In [67]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]

volume = ((1-tmask) * mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'depth', 'y':'gridY', 'x':'gridX'})
rho_inv = (1/1026)

# e3t_1d = mesh.e3t_1d.rename({'z':'depth'}).isel(t=0)
# e3t_3d = mesh.e3t_0.rename({'z':'depth', 'y':'gridY', 'x':'gridX'}).isel(t=0)
volume

<xarray.DataArray (depth: 40, gridY: 898, gridX: 398)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Dimensions without coordinates: depth, gridY, gridX

In [68]:
chunk_sizes = {'time': 1, 'depth': 40, 'gridY': 898, 'gridX': 398}

In [69]:
year = 2023
# fname = 'SalishSeaCast_river_trace_day_avg_allR_00_40_{}0101_{}1231_e3t_3d_vol'.format(year, year)
fname = 'SalishSeaCast_river_trace_day_avg_allR_00_40_{}0101_{}1231_e3t_3d_vol'.format(year, year)
fw = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_vol/{}.nc'.format(fname),chunks=chunk_sizes)

In [70]:
fw

<xarray.Dataset>
Dimensions:          (time: 365, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time             (time) datetime64[ns] 2023-01-01T12:00:00 ... 2023-12-31...
  * depth            (depth) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
  * gridY            (gridY) int64 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX            (gridX) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396 397
Data variables: (12/13)
    nooksack_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    skagit_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    snohomish_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    nisqually_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    elwha_river      (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    cowichan_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    ...               ...
    puntledge_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    salmon_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    homathko_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    squamish_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    fraser_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    other_rivers     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>

In [71]:
client=Client(processes=True, n_workers=4, local_directory='/tmp/cdonaldson')  # 4 is a good choice for n_workers even on salish, if I was the only person logged on or asked for space, could go as high as 16 but... be conscious of this
# threads_per_worker = 1 if needed, netcdf is not threadsafe now so be realiable by specifying 1 (at expense of just some efficiency)

In [72]:
# # vol = fw * e3t_1d
# vol = rho_inv * fw * volume
vol = fw

In [73]:
for data_var in list(vol.data_vars):
    vol[data_var].attrs['units'] = 'm^3'

In [74]:
vol_sum = vol.sum(dim='depth')

In [75]:
vol_sum

<xarray.Dataset>
Dimensions:          (time: 365, gridY: 898, gridX: 398)
Coordinates:
  * time             (time) datetime64[ns] 2023-01-01T12:00:00 ... 2023-12-31...
  * gridY            (gridY) int64 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX            (gridX) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396 397
Data variables: (12/13)
    nooksack_river   (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    skagit_river     (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    snohomish_river  (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    nisqually_river  (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    elwha_river      (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    cowichan_river   (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    ...               ...
    puntledge_river  (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    salmon_river     (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    homathko_river   (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    squamish_river   (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    fraser_river     (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>
    other_rivers     (time, gridY, gridX) float64 dask.array<chunksize=(1, 898, 398), meta=np.ndarray>

In [76]:
fname_save = fname + '_e3t_depsum_vol'
fname_save

'SalishSeaCast_river_trace_day_avg_allR_00_40_20230101_20231231_e3t_3d_vol_e3t_depsum_vol'

In [77]:
vol_sum.to_netcdf('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_vol/{}.nc'.format(fname_save), engine="netcdf4")
# vol.compute()

## shut down the client

In [78]:
client.close()